In [ ]:
"Créditos: La arquitectura de red neuronal Unet++ fue modificada del repositorio unetplusplus de Yihang Xin en su repositorio Github: https://github.com/yihangx/Unetplusplus/blob/master/Unet%2B%2B.ipynb"


In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.metrics import BinaryAccuracy, BinaryIoU
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import pandas as pd


In [ ]:
# Rutas de las carpetas
carpeta_fallas = "/kaggle/input/implem-fase2/fase2/Fallas"
carpeta_sismica = "/kaggle/input/implem-fase2/fase2/Sismica"

#import shutil

#local_fallas = "/content/local_fallas"
#local_sismica = "/content/local_sismica"

# Copiar carpetas localmente
#shutil.copytree(carpeta_fallas, local_fallas)
#shutil.copytree(carpeta_sismica, local_sismica)

# Actualizar las rutas
#carpeta_fallas = local_fallas
#carpeta_sismica = local_sismica

# Listar archivos en ambas carpetas
archivos_fallas = sorted(os.listdir(carpeta_fallas))
archivos_sismica = sorted(os.listdir(carpeta_sismica))

# Cargar y emparejar
fallas_data = []
sismica_data = []

for archivo_falla, archivo_sismica in zip(archivos_fallas, archivos_sismica):
    if archivo_falla == archivo_sismica:  # Verifica que los nombres coincidan
        # Cargar los arrays
        falla_array = np.load(os.path.join(carpeta_fallas, archivo_falla))
        sismica_array = np.load(os.path.join(carpeta_sismica, archivo_sismica))

        # Asegurarse de que las imágenes tengan la misma forma
        if falla_array.shape != (128, 128):
            falla_array = resize(falla_array, (128, 128))
        if sismica_array.shape != (128, 128):
            sismica_array = resize(sismica_array, (128, 128))

        # Añadir a las listas
        fallas_data.append(falla_array)
        sismica_data.append(sismica_array)
    else:
        print(f"Advertencia: Los archivos no coinciden: {archivo_falla} vs {archivo_sismica}")

images = np.array(sismica_data)
masks = np.array(fallas_data)



In [ ]:
#Dividir dataset
images_train, images_test, masks_train, masks_test = train_test_split(images, masks, random_state=13, test_size=0.2)
images_train = np.nan_to_num(images_train, nan=0.0)
images_test = np.nan_to_num(images_test, nan=0.0)

In [ ]:
#Visualizar los datos
def visualize_data(images, masks, num_samples=5):
    plt.figure(figsize=(12, 6))

    for i in range(num_samples):
        # Mostrar la imagen
        plt.subplot(2, num_samples, i + 1)
        plt.imshow(images[i+1].T, cmap='gray')
        plt.title(f'Imagen {i + 1}')
        plt.axis('off')

        # Mostrar la máscara
        plt.subplot(2, num_samples, i + 1 + num_samples)
        plt.imshow(masks[i+1].T, cmap='gist_gray')
        plt.title(f'Máscara {i + 1}')
        plt.axis('off')

    #plt.savefig("patches_unet++.svg", format="svg")
    plt.show()

visualize_data(images_train, masks_train, num_samples=4)

In [ ]:

dropout_rate = 0.1
activation = "relu"


In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (
    Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization,
    Activation, concatenate, Dropout
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf

# Definir conv_block
def conv_block(inputs, num_of_channels):
    x = Conv2D(num_of_channels, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    x = Dropout(0.1)(x)
    x = Conv2D(num_of_channels, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(x)
    return x

# Dimensiones de entrada
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 1

# Construcción del modelo
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = inputs

c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = Dropout(0.1)(c1)
c1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = MaxPooling2D((2, 2))(c1)


c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

up1_2 = Conv2DTranspose(32,(2,2),strides=(2,2),padding='same')(c2)
conv1_2 = concatenate([up1_2,c1],axis=3)
conv1_2 = conv_block(conv1_2, num_of_channels=32)

c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.1) (c3)
c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

up2_2 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
conv2_2 = concatenate([up2_2, c2], axis=3)
conv2_2 = conv_block(conv2_2, num_of_channels=64)

up1_3 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv2_2)
conv1_3 = concatenate([up1_3, c1, conv1_2], axis=3)
conv1_3 = conv_block(conv1_3, num_of_channels=32)

c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.1) (c4)
c4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

up3_2 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c4)
conv3_2 = concatenate([up3_2, c3], axis=3)
conv3_2 = conv_block(conv3_2, num_of_channels=128)

up2_3 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv3_2)
conv2_3 = concatenate([up2_3, c2, conv2_2], axis=3)
conv2_3 = conv_block(conv2_3, num_of_channels=64)

up1_4 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv2_3)
conv1_4 = concatenate([up1_4, c1, conv1_2, conv1_3], axis=3)
conv1_4 = conv_block(conv1_4, num_of_channels=32)

c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.1) (c5)
c5 = Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

up4_2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
conv4_2 = concatenate([up4_2, c4],  axis=3)
conv4_2 = conv_block(conv4_2,  num_of_channels=256)

up3_3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4_2)
conv3_3 = concatenate([up3_3, c3, conv3_2],  axis=3)
conv3_3 = conv_block(conv3_3, num_of_channels=128)

up2_4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv3_3)
conv2_4 = concatenate([up2_4, c2, conv2_2, conv2_3], axis=3)
conv2_4 = conv_block(conv2_4, num_of_channels=64)

up1_5 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv2_4)
conv1_5 = concatenate([up1_5, c1, conv1_2, conv1_3, conv1_4],  axis=3)
conv1_5 = conv_block(conv1_5, num_of_channels=32)

# Salida del modelo
nestnet_output = Conv2D(1, (1, 1), activation='sigmoid',
                        kernel_initializer='he_normal', padding='same')(conv1_5)
model = Model(inputs=inputs, outputs=nestnet_output)

# Compilar el modelo
optimizer = Adam(learning_rate=1e-4)
loss_fn = BinaryCrossentropy(from_logits=False)
metrics = [tf.keras.metrics.Precision(),BinaryIoU(target_class_ids=[1], threshold=0.5, name=None, dtype=None), BinaryAccuracy(dtype=None, threshold=0.5)]

model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

# Mostrar el resumen del modelo
model.summary()


In [ ]:
epochs = 25
batch_size = 32

In [ ]:
historia = model.fit(images_train, masks_train, epochs=epochs, batch_size=batch_size, shuffle=True,validation_split=0.2)

In [ ]:
#Metricas de entrenamiento
Loss = historia.history['loss']
IoU = historia.history['binary_io_u']
Precision = historia.history['precision']
Accuracy = historia.history['binary_accuracy']


#Métricas de validación
Val_Loss = historia.history['val_loss']
Val_IoU = historia.history['val_binary_io_u']
Val_Precision = historia.history['val_precision']
Val_Accuracy = historia.history['val_binary_accuracy']


In [ ]:
import csv
with open("training_metrics_++.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Loss", "IoU", "Precision", "Accuracy"])  # Encabezados correctos
    for loss, iou, precision, accuracy in zip(Loss, IoU, Precision, Accuracy):
        writer.writerow([loss, iou, precision, accuracy])

In [ ]:
#Grafica de Pérdida
epochs_range = range(1, len(Loss) + 1)
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, Loss, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_Loss, color='blue', label= 'Validacion')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.savefig("loss_unet++.svg", format="svg")
plt.show()

In [ ]:
#Gráfica de Precisión
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, Precision, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_Precision, color='blue', label='Validación')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('Precision')
plt.legend()
plt.savefig("precision_unet++.svg", format="svg")
plt.show()

In [ ]:
#Gráfica de IoU
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, IoU, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_IoU, color='blue', label='Validación')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('IoU')
plt.legend()
plt.savefig("IoU_unet++.svg", format="svg")
plt.show()


In [ ]:
#Gráfica de Accuracy
plt.figure(figsize=(7, 3))
plt.plot(epochs_range, Accuracy, color='red', label='Entrenamiento')
plt.plot(epochs_range, Val_Accuracy, color='blue', label='Validación')
plt.ylim(0,1)
plt.xlabel('Épocas')
plt.ylabel('Exactitud')
plt.legend()
plt.savefig("Accuracy_unet++.svg", format="svg")
plt.show()


In [ ]:
predicciones = model.predict(images_test)

In [ ]:
#Evaluar modelo con los datos de test.
historia2 = model.evaluate(
    x=images_test,
    y=masks_test,
    batch_size=None,
    verbose="auto",
    sample_weight=None,
    steps=None,
    callbacks=None,
    return_dict=False,)

In [ ]:
def mostrar_resultados(imagenes, mascaras_reales, mascaras_predichas, num_ejemplos=5):
    plt.figure(figsize=(15, num_ejemplos * 5))
    
    for i in range(num_ejemplos):
        # Imagen original
        plt.subplot(num_ejemplos, 3, i * 3 + 1)
        plt.imshow(imagenes[i].T, cmap="gray")
        plt.title("Imagen original")
        plt.axis("off")

        # Máscara real
        plt.subplot(num_ejemplos, 3, i * 3 + 2)
        plt.imshow(mascaras_reales[i].T, cmap="gray", interpolation='Nearest')
        plt.title("Máscara real")
        plt.axis("off")

        # Máscara predicha
        plt.subplot(num_ejemplos, 3, i * 3 + 3)
        plt.imshow(mascaras_predichas[i, ..., 0].T, cmap="gray", interpolation='Nearest')
        plt.title("Máscara predicha")
        plt.axis("off")

    plt.savefig("Resultado_UNet++.svg", format="svg")
    plt.tight_layout()
    plt.show()

# Mostrar resultados para las primeras 5 imágenes
mostrar_resultados(images_test, masks_test, predicciones, num_ejemplos=5)

In [ ]:
# Guardar el modelo
model.save("/kaggle/working/Unet++.keras")

# Cargar el modelo
from tensorflow.keras.models import load_model
modelo_cargado = load_model("/kaggle/working/Unet++.keras")

# Confirmar que el modelo cargado funciona.
modelo_cargado.summary()